## Modèle simple

In [1]:
from helpers import *

In [2]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]  
df = pd.read_csv("./../input/training.1600000.processed.noemoticon.csv", header=None, names=cols)

# df = df.sample(n=1000, random_state=42)
len_df = len(df)
print(df.shape)
df.head()

(1600000, 6)


,sentiment,timestamp,date,query,username,comment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [3]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 1600000/1600000 [00:03<00:00, 516323.39it/s]


(1469990, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,awww thats a bummer you shoulda got david carr...,19,16
1,is upset that he can't update his Facebook by ...,0,is upset that he cant update his facebook by t...,21,21
2,@Kenichan I dived many times for the ball. Man...,0,i dived many times for the ball managed to sav...,18,16
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,10,10
4,"@nationwideclass no, it's not behaving at all....",0,no its not behaving at all im mad why am i her...,21,20


In [4]:
pd.set_option('display.max_colwidth', None)  # Set max_colwidth to None to display full comments  
df[df["sentiment"] == 0]["comment"].head()

0    @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
1        is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!
2                              @Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds
3                                                                        my whole body feels itchy and like its on fire 
4        @nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there. 
Name: comment, dtype: object

In [5]:
df[df["sentiment"] == 1]["comment"].head()

800000                                                                                                 I LOVE @Health4UandPets u guys r the best!! 
800001                                                                     im meeting up with one of my besties tonight! Cant wait!!  - GIRL TALK!!
800002    @DaRealSunisaKim Thanks for the Twitter add, Sunisa! I got to meet you once at a HIN show here in the DC area and you were a sweetheart. 
800003                                     Being sick can be really cheap when it hurts too much to eat real food  Plus, your friends make you soup
800004                                                                                              @LovesBrooklyn2 he has that effect on everyone 
Name: comment, dtype: object

Cette approche consiste à entrainer les données avec une régression logistique.


In [9]:
# Copy original dataframe
df_simple_approach = df.copy()

X = df_simple_approach['comment_clean']
y = df_simple_approach['sentiment']

# Step 1: Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)  

# Step 2: Feature extraction
vectorizer = CountVectorizer()  
X_train_counts = vectorizer.fit_transform(X_train)  
X_test_counts = vectorizer.transform(X_test)  

print("CountVectorizer Feature Extraction")
print("X train before", X_train.shape)
print("X train after", X_train_counts.shape)

# Step 3: Train the model
print("\nTraining the model...")
model = LogisticRegression(solver='saga', max_iter=1000)
model.fit(X_train_counts, y_train)  
print("Training done")

# Step 4 predict
y_pred = model.predict(X_test_counts)  

print(f"\nAccuracy: {accuracy_score(y_test, y_pred)*100:.2f}%")  

CountVectorizer Feature Extraction
X train before (1175992,)
X train after (1175992, 350881)

Training the model...
Training done

Accuracy: 79.23%


### Export

In [7]:
# Save the trained model and CountVectorizer object  
with open('simple_sentiment_model.pkl', 'wb') as file:  
    pickle.dump(model, file)  
  
with open('simple_count_vectorizer.pkl', 'wb') as file:  
    pickle.dump(vectorizer, file)  

In [8]:
# Predict sentiments for the test set  
# model.predict(["I am not happy this is bad news"])  
local_test = vectorizer.transform(["i am so happy this is good news!"])
local_pred = model.predict(local_test)  
local_pred

array([1])